In [73]:
import pandas as pd
from ztfquery import lightcurve
import time
import numpy as np
from tqdm import tqdm
from tqdm.auto import tqdm
import astropy
from astropy import units as u
from astropy.coordinates import SkyCoord
from antares_client.search import get_by_id, get_by_ztf_object_id


calculate_flux = lambda x: 10**((x + 48.6) / -2.5)


cite = 'https://sites.astro.caltech.edu/ztf/bts/explorer.php?f=s&subsample=trans&classstring=&classexclude=&quality=y&purity=y&lcfig=y&showdiscdate=y&hostphot=y&ztflink=lasair&lastdet=&startsavedate=&startpeakdate=&startra=&startdec=&startz=&startdur=&startrise=&startfade=&startpeakmag=&startabsmag=&starthostabs=&starthostcol=&startb=&startav=&endsavedate=&endpeakdate=&endra=&enddec=&endz=&enddur=&endrise=&endfade=&endpeakmag=19.0&endabsmag=&endhostabs=&endhostcol=&endb=&endav=&sort=peakdate&format=csv'
path_to = "/home/masha/AstroAugumetation/ZTF_BTS_02_02__02_04_2021.csv"
bts = pd.read_csv(path_to, sep =',')
bts = bts.drop('Unnamed: 0', 1) 
bts

,ZTFID,IAUID,discdate,RA,Dec,peakt,peakfilt,peakmag,peakabs,duration,rise,fade,type,redshift,hostabs,hostcol,b,A_V
0,ZTF18aaqedfj,SN2018bff,2018-04-23 11:26:52.000,13:45:25.21,+26:25:27.2,242.77,r,18.1144,-19.09,27.8,4.738,23.062,SN Ia,0.0623,-19.90,0.96,77.936810,0.040
1,ZTF18aarbgdn,SN2018bfh,2018-05-06 03:50:24.000,10:22:41.10,+15:56:20.8,245.65,r,17.2666,-19.22,>16.002,3.922,>12.08,SN Ia,0.044,-22.20,0.97,53.801429,0.096
2,ZTF18aajpjdi,SN2018bfx,2018-04-14 07:24:57.000,13:18:06.62,+27:52:24.2,245.71,r,18.4652,-19.13,39.856,18.904,20.952,SN Ia,0.0746,-20.07,0.53,84.039406,0.040
3,ZTF18aaqjovh,SN2018bvw,2018-05-05 04:04:48.000,11:52:43.61,+25:40:30.1,248.74,r,18.3077,-18.58,25.566,8.684,16.882,SN Ic-BL,0.054,-19.99,0.62,76.780820,0.044
4,ZTF18aatlfus,SN2018clq,2018-06-08 12:15:50.000,17:08:42.33,+28:31:14.1,252.92,g,18.3244,-18.39,23.432,4.13,19.302,SN IIP,0.0451,-21.47,0.82,33.822947,0.213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2731,ZTF21aaopqqk,SN2021gbq,2021-03-07 09:50:24.000,18:41:50.46,+61:15:23.1,1292.01,g,18.2948,-19.30,20.285,8.549,11.736,SN Ia,0.0705,-19.00,0.60,24.690561,0.128
2732,ZTF21aaocrlm,SN2021epp,2021-03-05 04:30:27.000,08:10:55.27,-06:02:49.3,1292.69,r,18.8046,-17.43,>24.583,13.533,>11.05,SN Ic-BL,0.0385,-20.18,0.56,14.642573,0.150
2733,ZTF21aaovdlh,AT2021fck,2021-03-05 04:05:27.001,08:41:22.68,+02:53:23.3,1293.65,g,17.8715,-19.16,>16.227,6.137,>10.09,SN Ia,0.056,-17.72,0.49,25.671500,0.084
2734,ZTF21aaodljj,AT2021epq,2021-03-05 04:11:15.996,08:36:28.83,+74:09:58.9,1293.66,g,17.6395,-19.22,>18.083,9.013,>9.07,SN Ia,0.053,-19.83,0.87,33.101619,0.043


In [13]:
bts.columns

Index(['ZTFID', 'IAUID', 'discdate', 'RA', 'Dec', 'peakt', 'peakfilt',
       'peakmag', 'peakabs', 'duration', 'rise', 'fade', 'type', 'redshift',
       'hostabs', 'hostcol', 'b', 'A_V'],
      dtype='object')

In [14]:
len(bts['ZTFID'].unique())

2736

Все значения колонки ZTFID являются уникальными, поэтому можем искать по этим ID в базе данных, чтобы избежать проблем сопоставления по координатам!

In [40]:
df_obj_test = get_by_ztf_object_id('ZTF21aapecxb')

In [41]:
lc_test = df_obj_test.lightcurve
lc_test

,alert_id,ant_mjd,ant_survey,ant_ra,ant_dec,ant_passband,ant_mag,ant_magerr,ant_maglim,ant_mag_corrected,ant_magerr_corrected,ant_magulim_corrected,ant_magllim_corrected
0,ztf_upper_limit:ZTF21aapecxb-1506252955915,59260.252951,2,NaN,NaN,R,NaN,NaN,20.052999,NaN,NaN,NaN,NaN
1,ztf_upper_limit:ZTF21aapecxb-1506276735915,59260.276736,2,NaN,NaN,g,NaN,NaN,19.867800,NaN,NaN,NaN,NaN
2,ztf_upper_limit:ZTF21aapecxb-1509232895915,59263.232894,2,NaN,NaN,R,NaN,NaN,19.648001,NaN,NaN,NaN,NaN
3,ztf_upper_limit:ZTF21aapecxb-1509275005915,59263.275000,2,NaN,NaN,g,NaN,NaN,19.757900,NaN,NaN,NaN,NaN
4,ztf_upper_limit:ZTF21aapecxb-1511252735915,59265.252732,2,NaN,NaN,R,NaN,NaN,20.313000,NaN,NaN,NaN,NaN
5,ztf_upper_limit:ZTF21aapecxb-1513173555915,59267.173553,2,NaN,NaN,R,NaN,NaN,19.787600,NaN,NaN,NaN,NaN
6,ztf_upper_limit:ZTF21aapecxb-1513234295915,59267.234294,2,NaN,NaN,g,NaN,NaN,19.704500,NaN,NaN,NaN,NaN
7,ztf_upper_limit:ZTF21aapecxb-1516231675915,59270.231678,2,NaN,NaN,g,NaN,NaN,18.990499,NaN,NaN,NaN,NaN
8,ztf_upper_limit:ZTF21aapecxb-1516293435915,59270.293438,2,NaN,NaN,R,NaN,NaN,19.063900,NaN,NaN,NaN,NaN
9,ztf_upper_limit:ZTF21aapecxb-1518171845915,59272.171840,2,NaN,NaN,g,NaN,NaN,19.059099,NaN,NaN,NaN,NaN


In [43]:
lc_test.columns

Index(['alert_id', 'ant_mjd', 'ant_survey', 'ant_ra', 'ant_dec',
       'ant_passband', 'ant_mag', 'ant_magerr', 'ant_maglim',
       'ant_mag_corrected', 'ant_magerr_corrected', 'ant_magulim_corrected',
       'ant_magllim_corrected'],
      dtype='object')

In [44]:
lc_test = lc_test.drop(['alert_id', 'ant_survey', 'ant_ra', 'ant_dec', \
                        'ant_maglim', 'ant_mag_corrected', 'ant_magerr_corrected', \
                        'ant_magulim_corrected', 'ant_magllim_corrected'], 1)

In [45]:
lc_test

,ant_mjd,ant_passband,ant_mag,ant_magerr
0,59260.252951,R,NaN,NaN
1,59260.276736,g,NaN,NaN
2,59263.232894,R,NaN,NaN
3,59263.275000,g,NaN,NaN
4,59265.252732,R,NaN,NaN
5,59267.173553,R,NaN,NaN
6,59267.234294,g,NaN,NaN
7,59270.231678,g,NaN,NaN
8,59270.293438,R,NaN,NaN
9,59272.171840,g,NaN,NaN


In [46]:
lc_test = lc_test.dropna()

In [47]:
lc_test.to_csv('ZTF21aapecxb.csv')

In [59]:
path = '/home/masha/AstroAugumetation/ANTARES/'
start_time = time.time()
for ztf_id in tqdm(bts['ZTFID'].values[248:]):
    df_obj = get_by_ztf_object_id(ztf_id)
    if df_obj is None:
        continue
    df_lc = df_obj.lightcurve
    df_lc = df_lc[['ant_mjd', 'ant_passband', 'ant_mag', 'ant_magerr']]
    df_lc = df_lc.dropna()
    df_lc.to_csv(path + ztf_id + '.csv')
    
end = time.time() - start_time
print(end/3600)

  0%|          | 0/2488 [00:00<?, ?it/s]

1.4883170045084424


In [95]:
data = []
new_bts_id = []
for ztf_id in tqdm(bts['ZTFID'].values):
    try:
        df = pd.read_csv(path + ztf_id + '.csv')
        if df.index.values.shape[0] != 0:
            df = df.drop('Unnamed: 0', 1)
            mask_g = df[df['ant_passband'] == 'g'].index.shape[0] > 10
            mask_r = df[df['ant_passband'] == 'R'].index.shape[0] > 10
            if (mask_g)&(mask_r):
                df = df[df['ant_passband'] != 'i']
                df['ant_mag'] = 1e26*df['ant_mag'].map(calculate_flux) #* 1e26 # переводим в потоки в [мЯн]
                df['ant_magerr'] = 1.086*df['ant_magerr']*df['ant_mag']
                df['ant_passband'][df['ant_passband'] == 'g'] = 0
                df['ant_passband'][df['ant_passband'] == 'R'] = 1
                df.rename(columns={"ant_mjd": "mjd","ant_mag": "flux", \
                                   "ant_magerr": "flux_err", "ant_passband": "passband"},
                          inplace=True)
                data.append(df)
                new_bts_id.append(ztf_id)
    except FileNotFoundError:
        continue

  0%|          | 0/2736 [00:00<?, ?it/s]

<ipython-input-95-c6f3352a5168>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ant_passband'][df['ant_passband'] == 'g'] = 0
<ipython-input-95-c6f3352a5168>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ant_passband'][df['ant_passband'] == 'R'] = 1


In [96]:
len(data)

1403

In [97]:
data[0] #['ant_passband' == 'g']

,mjd,passband,flux,flux_err
0,58242.252847,1,0.141579,0.004256
1,58245.207500,1,0.149252,0.022284
2,58248.252627,1,0.108663,0.005077
3,58248.254537,1,0.121071,0.004436
4,58255.232431,1,0.098157,0.006649
...,...,...,...,...
204,59290.437859,0,0.089617,0.009449
205,59291.303449,0,0.034325,0.006865
206,59297.304745,0,0.070664,0.020601
207,59303.359861,0,0.093496,0.019217


In [98]:
len(new_bts_id)

1403

In [99]:
new_bts_id[0]

'ZTF18aajpjdi'